# Accident Report System

In [29]:
# Initial imports
import sys

# this is importing from the crypto.py file we created in the same folder
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3

# if not installed, need to run:
# !pip install prettyprinter 
from pprint import pprint

### Creating an instance of the contract that we want to interact with

In [30]:
cryptofax = initContract()

### Defining the functions we want to use

In [31]:
def createAccidentReport():
    time = input("Date of the accident: ")
    description = input("Description of the accident: ")
    token_id = int(input("Token ID: "))

    json_data = convertDataToJSON(time, description)
    report_uri = pinJSONtoIPFS(json_data)

    return token_id, report_uri


In [32]:
def reportAccident(token_id, report_uri):
    tx_hash = cryptofax.functions.reportAccident(token_id, report_uri).transact(
        {"from": w3.eth.accounts[0]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [33]:
def getAccidentReports(token_id):
    accident_filter = cryptofax.events.Accident.createFilter(
        fromBlock="0x0", argument_filters={"token_id": token_id}
    )
    return accident_filter.get_all_entries()

### Suppose we we want to create a report:

In [ ]:
token_id, report_uri = createAccidentReport()

receipt = reportAccident(token_id, report_uri)

In [ ]:
pprint(receipt)

In [ ]:
print("Report IPFS Hash:", report_uri)

### Suppose we we want to pull information on previously reported accidents:

In [ ]:
token_id = int(sys.argv[2])

In [ ]:
car = cryptofax.functions.cars(token_id).call()

In [ ]:
reports = getAccidentReports(token_id)

In [ ]:
pprint(reports)

In [ ]:
print("VIN", car[0], "has been in", car[1], "accidents.")

### How do we make this accessible for non-programmers?

Suppose we wanted to allow others (who might not be familiar with Python) to conveniently run this from the command line (similar to how we used geth without knowing how to program in Go). We can do so by saving the file as a Python file (ending in .py, accident.py is already created) and creating a main() function and calling it so that whenever the entire file gets called, main() gets executed:

In [41]:
def main():
    if sys.argv[1] == "report":
        token_id, report_uri = createAccidentReport()

        receipt = reportAccident(token_id, report_uri)

        pprint(receipt)
        print("Report IPFS Hash:", report_uri)

    if sys.argv[1] == "get":
        token_id = int(sys.argv[2])

        car = cryptofax.functions.cars(token_id).call()
        reports = getAccidentReports(token_id)

        pprint(reports)
        print("VIN", car[0], "has been in", car[1], "accidents.")

In [40]:
main()

### A brief intro to sys.argv:

sys.argv is the list of arguments (or flags) passed from the command line by the user. We used a similar functionality when interacting with the puppeth tool using double dash (e.g.  "./geth account new --datadir node1"
```
sys.argv[0] is always the name of the script

sys.argv[1] is the first argument, and so on
```
 Below is an example:

In [39]:
#        sys.argv[0]        sys.argv[1]    sys.argv[2]
# python accident.py        report
# python accident.py        get            1

That is, if we want to create a report we run this from the command line (after nagivating into the folder why accidents.py is saved):

python accident.py report

If we want to pull information on previously reported accidents, we run:

python accident.py get 1